In [ ]:
from qdrant_client import QdrantClient, models
import numpy as np
from fastembed import TextEmbedding

/workspaces/llm-zoomcamp-dm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [72]:
qd_client = QdrantClient(
    url="http://localhost:6333")

## Question 1.

In [17]:
# Choosing the embedding model : # jinaai/jina-embeddings-v2-small-en
embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')

In [22]:
# initializing the query
# q = "I just discovered the course. Can I join now?" and embedding it using the embedding model chosen
query = "I just discovered the course. Can I join now?"
q = np.array(list(embedding_model.embed([query])))[0]
q.shape

(512,)

In [23]:
print(f"The minimum value in the array is: {float(min(q))}")

The minimum value in the array is: -0.11726373885183883


In [29]:
import numpy as np
float(np.linalg.norm(q))

1.0

In [30]:
float(q.dot(q))

1.0000000000000002

## Question 2

In [31]:
# initializing the doc 
doc = 'Can I still join the course after the start date?'

In [32]:
# converting this doc into a vector using the embedding model chosen.
doc_vector = np.array(list(embedding_model.embed([doc])))[0]
doc_vector.shape

(512,)

In [37]:
# calculating the cosine similarity between the query and the doc vector
cosine_similarity = doc_vector.dot(q)
print(f"The cosine similarity between the query vector and doc vector is: {float(cosine_similarity)}")

The cosine similarity between the query vector and doc vector is: 0.9008528895674548


## Question 3

In [38]:
# initializing the documents
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [ ]:
index = 0
for document in documents:
    temp_document_vector = np.array(list(embedding_model.embed(document['text'])))[0]
    print(f"index: {index} and cosine similarity: {q.dot(temp_document_vector)}")
    index += 1

index: 0 and cosine similarity: 0.7629684696540238
index: 1 and cosine similarity: 0.8182378150042889
index: 2 and cosine similarity: 0.8085397398734189
index: 3 and cosine similarity: 0.7133079015686243
index: 4 and cosine similarity: 0.7304499234333598


In [ ]:
# trying the matrix method
embeddings_text = []
for document in documents:
    embeddings_text.append(np.array(list(embedding_model.embed(document['text'])))[0])


list

## Question 4

In [50]:
index = 0
for document in documents:
    temp_document_vector = np.array(list(embedding_model.embed(document['question'] + ' ' + document['text'])))[0]
    print(f"index: {index} and cosine similarity: {q.dot(temp_document_vector)}")
    index += 1

index: 0 and cosine similarity: 0.8514543236908068
index: 1 and cosine similarity: 0.8436594159113068
index: 2 and cosine similarity: 0.8408287048502558
index: 3 and cosine similarity: 0.7755157969663908
index: 4 and cosine similarity: 0.8086007795043938


The answer for Q4 is more appropriate as the question in the 0th indexed document closely matches the query vector. Hence giving rise to a higher cosine similarity score

## Question 5

In [ ]:
# checking the dimensions of all the supported models in fastembed
model_dimensions = []
for model in TextEmbedding.list_supported_models():
    model_dimensions.append(int(model['dim']))
min_dim = min(model_dimensions)

In [63]:
print(f"The smallest dimension in the supported fastembed models is: {min_dim}")

The smallest dimension in the supported fastembed models is: 384


In [62]:
for model in TextEmbedding.list_supported_models():
    if model['dim'] == min_dim:
        print(model['model'])

BAAI/bge-small-en
BAAI/bge-small-en-v1.5
snowflake/snowflake-arctic-embed-xs
snowflake/snowflake-arctic-embed-s
sentence-transformers/all-MiniLM-L6-v2
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


In [66]:
new_embedding_model_name = "BAAI/bge-small-en"

In [67]:
new_embedding_model = TextEmbedding(model_name=new_embedding_model_name)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00,  7.41it/s]


## Question 6

In [68]:
# Bringing in more documents
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [70]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [73]:
# adding them a collection in qdrant
collection_name = "zoomcamp-vector-search"

qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_dim,
        distance=models.Distance.COSINE
    )
)

True

In [74]:
points = []
id=0

for doc in documents:
    text= doc['question'] + ' ' + doc['text']
    point=models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=new_embedding_model_name),
        payload={
            "text":doc["text"],
            "question":doc["question"],
            "course":doc["course"],
            "section":doc["section"]
        }
    )
    points.append(point)
    id += 1

In [ ]:
points[0]

In [76]:
# uploading this collection onto qdrant
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [78]:
def search(query, limit):
    results=qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=new_embedding_model_name
        ),
        limit=limit,
        with_payload=True
    )
    return results

In [88]:
results = search(query,5).points
print(f"Question: {query}")
for result in results:
    print()
    print(f"Answer: \n{result.payload['text']} \n- score {result.score}")

Question: I just discovered the course. Can I join now?

Answer: 
Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate. 
- score 0.8703172

Answer: 
Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc) 
- score 0.86918855

Answer: 
Here’s how you join a in Slack: https://slack.com/help/articles/205239967-Join-a-channel
Click “All channels” at the top of your left sidebar. If you don't see this option, click “More” to find it.
Browse the list of public channels in your workspace, or use the search bar to search by channel name or description.
Select a channel from the list to view it.
Click Join Channel.
Do we